In [1]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import glob
import numpy as np
import os

In [3]:
class load_balanced_view_replace:
    
    def apply(self, func, *args):
        func(*args)
    
def func(a,b):
    print(a,b)
    
lv = load_balanced_view_replace()
lv.apply(func, 1, 2)

1 2


In [4]:
try:
    # have IPython.parallel?
    
    from IPython.parallel import Client
    c = Client()
    lv = c.load_balanced_view()   # this object represents the engines (workers)
    print("Using IPython.parallel.")
except:
    class task:
        def __init__(self, result):
            self.result = result
            
        def get(self):
            return self.result
        
    # else, execute commands in sequence
    class load_balanced_view_replace:
    
        def apply(self, func, *args):
            return task( func(*args) )
    
    lv = load_balanced_view_replace()

/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json


In [5]:
field='COSMOSA'

In [6]:
def filter_cube(f, DRYRUN=True):
    import os
    h,t = os.path.split(f)
    cmd="python filter_cube.py --infile {} --outfile {}".format(f,os.path.join(h,"sf2" + t) )
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

In [7]:
# filter data cubes
ff = np.sort( glob.glob("../data/outcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(filter_cube, f))
result = [task.get() for task in tasks]  # blocks until all results are back

python filter_cube.py --infile ../data/outcube_COSMOSA_022.fits.gz --outfile ../data/sf2outcube_COSMOSA_022.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_023.fits.gz --outfile ../data/sf2outcube_COSMOSA_023.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_032.fits.gz --outfile ../data/sf2outcube_COSMOSA_032.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_033.fits.gz --outfile ../data/sf2outcube_COSMOSA_033.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_034.fits.gz --outfile ../data/sf2outcube_COSMOSA_034.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_035.fits.gz --outfile ../data/sf2outcube_COSMOSA_035.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_036.fits.gz --outfile ../data/sf2outcube_COSMOSA_036.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_037.fits.gz --outfile ../data/sf2outcube_COSMOSA_037.fits.gz
python filter_cube.py --infile ../data/outcube_COSMOSA_042.fits.gz --out

In [8]:
#build noise model
def noise_model(f, DRYRUN=True):
    import os
    h,t = os.path.split(f)
    cmd="python noise_model2.py --infile {} --outfile {}".format(f,os.path.join(h,"nc" + t) )
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

tasks = []
for f in ff:
    tasks.append(lv.apply(noise_model, f))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

python noise_model2.py --infile ../data/outcube_COSMOSA_022.fits.gz --outfile ../data/ncoutcube_COSMOSA_022.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_023.fits.gz --outfile ../data/ncoutcube_COSMOSA_023.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_032.fits.gz --outfile ../data/ncoutcube_COSMOSA_032.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_033.fits.gz --outfile ../data/ncoutcube_COSMOSA_033.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_034.fits.gz --outfile ../data/ncoutcube_COSMOSA_034.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_035.fits.gz --outfile ../data/ncoutcube_COSMOSA_035.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_036.fits.gz --outfile ../data/ncoutcube_COSMOSA_036.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_037.fits.gz --outfile ../data/ncoutcube_COSMOSA_037.fits.gz
python noise_model2.py --infile ../data/outcube_COSMOSA_042.fits.gz --ou

In [11]:
# filter noise cubes
ff = np.sort( glob.glob("data/ncoutcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(filter_cube, f))
result = [task.get() for task in tasks]  # blocks until all results are back

# don't need the unsmoothed noise cubes anymore
cmd="rm data/ncoutcube_{}_???.fits.gz".format(field)

In [12]:
detect_limits = {}

detect_limits['COSMOSA'] = { 'detect_threshold':0.35, 'grow_threshold':0.21}
detect_limits['COSMOSB'] = { 'detect_threshold':0.64, 'grow_threshold':0.39}
detect_limits['COSMOSC'] = { 'detect_threshold':0.64, 'grow_threshold':0.39}
detect_limits['COSMOSD'] = { 'detect_threshold':0.64, 'grow_threshold':0.39}

In [13]:
if False:
    # run detect and segmentation
    def cloud_finder2(f, field, detect_limits):
        import os
        h,t = os.path.split(f)
        dt, gt = detect_limits[field]['detect_threshold'], detect_limits[field]['grow_threshold']
        cmd = "python cloud_finder2.py -i {} --detect_threshold {} --grow_threshold {} -b {} -o {}".format(f,dt,gt,f.replace('.fits.gz','.badregs'),f.replace('sf2outcube','map'))
        return cmd
        #return os.system( cmd )

    ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

    tasks = []
    for f in ff:
        tasks.append(lv.apply(cloud_finder2, f, field, detect_limits))
    result = [task.get() for task in tasks]  # blocks until all results are back

    for r in result:
        print(r)

In [226]:
# run detect and segmentation
def build_detect_noise_model(f, DRYRUN=True):
    import os
    detect_noise_model = f.replace(".fits.gz",".detect_noise_model")
    cmd = "python detection_noise_model.py -i {} -o {}".format(f,detect_noise_model)
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(build_detect_noise_model, f))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
   print(r)

(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_021.fits.gz -o data/sf2outcube_COSMOSC_021.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_022.fits.gz -o data/sf2outcube_COSMOSC_022.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_023.fits.gz -o data/sf2outcube_COSMOSC_023.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_024.fits.gz -o data/sf2outcube_COSMOSC_024.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_025.fits.gz -o data/sf2outcube_COSMOSC_025.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_026.fits.gz -o data/sf2outcube_COSMOSC_026.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_027.fits.gz -o data/sf2outcube_COSMOSC_027.detect_noise_model')
(-1, 'python detection_noise_model.py -i data/sf2outcube_COSMOSC_032.fits.gz -o data/sf2outcube_C

In [228]:
sigma_detect_threshold = 5.
sigma_grow_threshold = 3.

# run detect and segmentation
def cloud_finder3(f, field, sigma_detect_threshold, sigma_grow_threshold, DRYRUN=True):
    import os
    h,t = os.path.split(f)
    detect_noise_model = f.replace(".fits.gz",".detect_noise_model")
    
    cmd = "python cloud_finder3.py -i {} --sigma_detect_threshold {} --sigma_grow_threshold {} -b {} -o {} -n {}".\
        format(f,sigma_detect_threshold, sigma_grow_threshold,f.replace('.fits.gz','.badregs'),f.replace('sf2outcube','map'),detect_noise_model)
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd
    
    return os.system( cmd )

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(cloud_finder3, f, field, sigma_detect_threshold, sigma_grow_threshold))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

(-1, 'python cloud_finder3.py -i data/sf2outcube_COSMOSC_021.fits.gz --sigma_detect_threshold 5.0 --sigma_grow_threshold 3.0 -b data/sf2outcube_COSMOSC_021.badregs -o data/map_COSMOSC_021.fits.gz -n data/sf2outcube_COSMOSC_021.detect_noise_model')
(-1, 'python cloud_finder3.py -i data/sf2outcube_COSMOSC_022.fits.gz --sigma_detect_threshold 5.0 --sigma_grow_threshold 3.0 -b data/sf2outcube_COSMOSC_022.badregs -o data/map_COSMOSC_022.fits.gz -n data/sf2outcube_COSMOSC_022.detect_noise_model')
(-1, 'python cloud_finder3.py -i data/sf2outcube_COSMOSC_023.fits.gz --sigma_detect_threshold 5.0 --sigma_grow_threshold 3.0 -b data/sf2outcube_COSMOSC_023.badregs -o data/map_COSMOSC_023.fits.gz -n data/sf2outcube_COSMOSC_023.detect_noise_model')
(-1, 'python cloud_finder3.py -i data/sf2outcube_COSMOSC_024.fits.gz --sigma_detect_threshold 5.0 --sigma_grow_threshold 3.0 -b data/sf2outcube_COSMOSC_024.badregs -o data/map_COSMOSC_024.fits.gz -n data/sf2outcube_COSMOSC_024.detect_noise_model')
(-1, 'py

In [213]:
# add segmants for manually flagged 
def apply_manual_catalog(f, field, sigma_grow_threshold, DRYRUN=False):
    import os
    h,t = os.path.split(f)
    ifu = t.replace('.fits.gz','').split("_")[-1]
    fncatalog = "data/{}_{}.txt".format(field, ifu)
    detect_noise_model = f.replace(".fits.gz",".detect_noise_model")
    
    cmd = "python apply_manual_catalog.py -i {} -s {} -m {} -n {} -o {} -c {}".\
        format(f, sigma_grow_threshold, f.replace('sf2outcube','map'), \
               detect_noise_model, f.replace('sf2outcube','mmap'), fncatalog)
    if DRYRUN:
        print(cmd)
        return cmd
    else:
        return os.system( cmd ), cmd
    
    return os.system( cmd )

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

sigma_grow_threshold = 2.0

tasks = []
for f in ff:
    tasks.append(lv.apply(apply_manual_catalog, f, field, sigma_grow_threshold))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

#python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_021.fits.gz --sigma_grow_threshold 1.7  
#-m data/map_COSMOSC_021.fits.gz -n data/sf2outcube_COSMOSC_021.detect_noise_model -o data/mmap_COSMOSC_021.fits.gz -c data/COSMOSC_021.txt

(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_021.fits.gz -s 2.0 -m data/map_COSMOSC_021.fits.gz -n data/sf2outcube_COSMOSC_021.detect_noise_model -o data/mmap_COSMOSC_021.fits.gz -c data/COSMOSC_021.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_022.fits.gz -s 2.0 -m data/map_COSMOSC_022.fits.gz -n data/sf2outcube_COSMOSC_022.detect_noise_model -o data/mmap_COSMOSC_022.fits.gz -c data/COSMOSC_022.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_023.fits.gz -s 2.0 -m data/map_COSMOSC_023.fits.gz -n data/sf2outcube_COSMOSC_023.detect_noise_model -o data/mmap_COSMOSC_023.fits.gz -c data/COSMOSC_023.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_024.fits.gz -s 2.0 -m data/map_COSMOSC_024.fits.gz -n data/sf2outcube_COSMOSC_024.detect_noise_model -o data/mmap_COSMOSC_024.fits.gz -c data/COSMOSC_024.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_025.fits.gz -s 2.0 -m data/map_COSMOSC_025.fi

In [223]:
#sf2outcube_COSMOSA_044.fits.gz
#map_COSMOSB_042.fits.gz


# build catalog
def build_catalog(f, field, DRYRUN=False):
    import os
    ifu = f[-11:-8]
    
    cmd = "python build_catalog.py -i {f} -m data/mmap_{field}_{ifu}.fits.gz -c specphot/mean_cal_{field}.txt -o data/msf2outcube_{field}_{ifu}.cat"\
        .format(f=f,field=field,ifu=ifu)
    
    if DRYRUN:
        return -1, cmd
    
    return os.system( cmd ), cmd

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(build_catalog, f, field))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

(0, 'python build_catalog.py -i data/sf2outcube_COSMOSC_021.fits.gz -m data/mmap_COSMOSC_021.fits.gz -c specphot/mean_cal_COSMOSC.txt -o data/msf2outcube_COSMOSC_021.cat')
(0, 'python build_catalog.py -i data/sf2outcube_COSMOSC_022.fits.gz -m data/mmap_COSMOSC_022.fits.gz -c specphot/mean_cal_COSMOSC.txt -o data/msf2outcube_COSMOSC_022.cat')
(0, 'python build_catalog.py -i data/sf2outcube_COSMOSC_023.fits.gz -m data/mmap_COSMOSC_023.fits.gz -c specphot/mean_cal_COSMOSC.txt -o data/msf2outcube_COSMOSC_023.cat')
(0, 'python build_catalog.py -i data/sf2outcube_COSMOSC_024.fits.gz -m data/mmap_COSMOSC_024.fits.gz -c specphot/mean_cal_COSMOSC.txt -o data/msf2outcube_COSMOSC_024.cat')
(0, 'python build_catalog.py -i data/sf2outcube_COSMOSC_025.fits.gz -m data/mmap_COSMOSC_025.fits.gz -c specphot/mean_cal_COSMOSC.txt -o data/msf2outcube_COSMOSC_025.cat')
(0, 'python build_catalog.py -i data/sf2outcube_COSMOSC_026.fits.gz -m data/mmap_COSMOSC_026.fits.gz -c specphot/mean_cal_COSMOSC.txt -o dat

In [224]:
# add segmants for manually flagged 
def add_manual_info(f, field, DRYRUN=True):
    import os
    h,t = os.path.split(f)
    ifu = t.replace('.fits.gz','').split("_")[-1]
    
    fncatalog = "data/{}_{}.txt".format(field, ifu)

    #python src/add_manual_info.py -m data/mmap_COSMOSC_021.fits.gz 
    # -c data/COSMOSC_021.txt -a data/msf2outcube_COSMOSC_021.cat
    
    cmd = "python src/add_manual_info.py -m {} -c {} -a {}".\
        format(f.replace('sf2outcube','mmap'), fncatalog, f.replace('.fits.gz','.cat').replace("sf2outcube","msf2outcube"))
    if DRYRUN:
        print(cmd)
        return cmd
    else:
        return os.system( cmd ), cmd
    
    return os.system( cmd )

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(add_manual_info, f, field))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

python src/add_manual_info.py -m data/mmap_COSMOSC_021.fits.gz -c data/COSMOSC_021.txt -a data/msf2outcube_COSMOSC_021.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_022.fits.gz -c data/COSMOSC_022.txt -a data/msf2outcube_COSMOSC_022.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_023.fits.gz -c data/COSMOSC_023.txt -a data/msf2outcube_COSMOSC_023.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_024.fits.gz -c data/COSMOSC_024.txt -a data/msf2outcube_COSMOSC_024.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_025.fits.gz -c data/COSMOSC_025.txt -a data/msf2outcube_COSMOSC_025.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_026.fits.gz -c data/COSMOSC_026.txt -a data/msf2outcube_COSMOSC_026.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_027.fits.gz -c data/COSMOSC_027.txt -a data/msf2outcube_COSMOSC_027.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_032.fits.gz -c data/COSMOSC_032.txt -a data/msf2outcube_COSMOSC_032.cat
python src/add_m

In [ ]:

#combine (vstack) all tables from this field
from astropy.table import vstack, Column
from astropy.io import ascii

ff = glob.glob("data/msf2outcube_{field}_???.cat".format(field=field))
#print(ff)
tt = []
for f in ff:
    ifu = f[-7:-4]
    print(ifu)
    t = ascii.read(f, format="ecsv")
    t.add_column(Column([ifu]*len(t),name='ifu'),0)
    tt.append(t)
 
tall = vstack(tt)
tall.write("data/msf2outcube_{field}_allifu.cat".format(field=field),format="ascii.ecsv")   


In [236]:
tall = vstack(tt)
tall.write("data/msf2outcube_{field}_allifu.cat".format(field=field),format="ascii.ecsv")   


In [182]:
halos = (tall["class"] == "halo" ).data